# Installation Guide for Required Packages

To run the script successfully, you need to install the following packages:

- **auto-gptq**
- **datasets**
- **torch**
- **transformers**

You can install all of these packages with the following `pip` command:

```bash
pip install auto-gptq datasets torch transformers


In [2]:
import random

from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from datasets import load_dataset
import torch
from transformers import AutoTokenizer

c:\Users\BS567\Documents\Purple LLAMA\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension not installed.
CUDA extension not installed.


In [3]:
device = "cuda"
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
out_dir = model_id + "-GPTQ"

In [4]:
quantize_config = BaseQuantizeConfig(
    bits=4,
    group_size=128,
    damp_percent=0.01,
    desc_act=False
)

In [5]:
model = AutoGPTQForCausalLM.from_pretrained(model_id, quantize_config,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.to(device)

c:\Users\BS567\Documents\Purple LLAMA\.venv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LlamaGPTQForCausalLM(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 2048)
      (layers): ModuleList(
        (0-21): 22 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (k_proj): Linear(in_features=2048, out_features=256, bias=False)
            (v_proj): Linear(in_features=2048, out_features=256, bias=False)
            (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
            (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
            (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
          (post_attention_

In [6]:
n_samples = 100
data = load_dataset("allenai/c4", data_files="en/c4-train.00001-of-01024.json.gz", split=f"train[:{n_samples*5}]")
tokenized_data = tokenizer("\n\n".join(data['text']), return_tensors='pt')

# Format tokenized examples
examples_ids = []
for _ in range(n_samples):
    i = random.randint(0, tokenized_data.input_ids.shape[1] - tokenizer.model_max_length - 1)
    j = i + tokenizer.model_max_length
    input_ids = tokenized_data.input_ids[:, i:j]
    attention_mask = torch.ones_like(input_ids)
    examples_ids.append({'input_ids': input_ids, 'attention_mask': attention_mask})

c:\Users\BS567\Documents\Purple LLAMA\.venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\BS567\.cache\huggingface\hub\datasets--allenai--c4. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 356318 examples [00:06, 55948.62 examples/s]
Token indices sequence length is longer tha

In [7]:
%%time

# Quantize with GPTQ
model.quantize(
    examples_ids,
    batch_size=1,
    use_triton=True,
)

WARNING - triton is not installed, reset use_triton to False
INFO - Start quantizing layer 1/22
INFO - Quantizing self_attn.k_proj in layer 1/22...
INFO - Quantizing self_attn.v_proj in layer 1/22...
INFO - Quantizing self_attn.q_proj in layer 1/22...
INFO - Quantizing self_attn.o_proj in layer 1/22...
INFO - Quantizing mlp.up_proj in layer 1/22...
INFO - Quantizing mlp.gate_proj in layer 1/22...
INFO - Quantizing mlp.down_proj in layer 1/22...
INFO - Start quantizing layer 2/22
INFO - Quantizing self_attn.k_proj in layer 2/22...
INFO - Quantizing self_attn.v_proj in layer 2/22...
INFO - Quantizing self_attn.q_proj in layer 2/22...
INFO - Quantizing self_attn.o_proj in layer 2/22...
INFO - Quantizing mlp.up_proj in layer 2/22...
INFO - Quantizing mlp.gate_proj in layer 2/22...
INFO - Quantizing mlp.down_proj in layer 2/22...
INFO - Start quantizing layer 3/22
INFO - Quantizing self_attn.k_proj in layer 3/22...
INFO - Quantizing self_attn.v_proj in layer 3/22...
INFO - Quantizing self_a

CPU times: total: 1h 20min 4s
Wall time: 4h 8min 12s


In [8]:
# Save model and tokenizer
model.save_quantized(out_dir, use_safetensors=True)
tokenizer.save_pretrained(out_dir)

('TinyLlama/TinyLlama-1.1B-Chat-v1.0-GPTQ\\tokenizer_config.json',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-GPTQ\\special_tokens_map.json',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-GPTQ\\tokenizer.model',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-GPTQ\\added_tokens.json',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-GPTQ\\tokenizer.json')

In [9]:
del model

## Inference

In [10]:
import time

def time_it(start,end):
    nano = end-start
    return nano/1e9

max_token = 100

In [11]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Reload model and tokenizer
model = AutoGPTQForCausalLM.from_quantized(
    out_dir,
    device=device,
    use_triton=True,
    use_safetensors=True,
)
tokenizer = AutoTokenizer.from_pretrained(out_dir)

WARNING - Triton is not installed, reset use_triton to False.
WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
WARNING - ignoring unknown parameter in quantize_config.json: quant_method.
INFO - The layer lm_head is not quantized.


In [12]:
text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt").to(device)
start = time.time_ns()
outputs = model.generate(**inputs, max_new_tokens=max_token)
end = time.time_ns()
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
t = time_it(start,end)
print("Seconds:",t)
print("Token/s",len(outputs[0])/t)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Hello my name is John Smith and I am a software engineer. I have been working in the industry for the past 10 years. I have worked on various projects and have gained experience in various technologies. I am proficient in Java, Python, and C++ and have experience with various frameworks such as Django, Flask, and React. I have also worked on various web applications using technologies such as HTML, CSS, and JavaScript. I am a team player and have worked on various projects with
Seconds: 46.6444157
Token/s 2.2510733262331333


In [13]:
print(f"Model size: {model.get_memory_footprint():,} bytes")

Model size: 768,062,336 bytes
